# CS231n - Day5 (Lec10)

## Recurrent Neural Networks (RNN)

* Original NN had only one input and one output. We can give it more flexibility by allowing it to have diverse lengths of input/output. Where _'one'_ means fixed size input/output, and _'many'_ means variably sized input/output.
    * e.g. Image Captioning (one to many)
    * e.g. Sentiment classification (many to one)
    * e.g. Video classification (many to many)
    * e.g. Translation (sequence to sequence)


* But even without that flexible property, RNN is effective for one to one models!
    * e.g. Make the classifier make its decision by following a series of 'glimpses', and we can make them generate a new image using that path!
    
    
## Structure of RNN
* Each RNN core block has an *"internal state"* that produces output with its parameters.  

$$
x \rightarrow \left[ RNN \right] \rightarrow y
$$
<br>
* Every time an internal state reads an input, it is updated by a certain formula $f_W$.  

    $$
    h_t = f_W(h_{t-1}, x_t)
    $$
    <br>
    in vanilla RNN,  

    $$
    \begin{matrix}
    h_t & = & tanh(W_{hh}h_{t-1}+W_{xh}x_t) \\
        & = & tanh \left( (W_{hh} W_{hx}) \left( \begin{matrix} h_{t-1} \\ x_t \end{matrix} \right) \right) \\
        & = & tanh\left(W \left( \begin{matrix} h_{t-1}\\ x_t \end{matrix} \right)\right)
    \end{matrix}
    $$
    <br>
    so each input gets a different hidden state as it updates itself constantly.

    We can also produce an output each time.
    $$
    y_t = W_{hy}h_t
    $$
    <br>

    Each $W_{hh}, W_{xh}, W_{hy}$ stay the same in one forward propagation, and is updated with backpropagation after the whole graph is unrolled.


* Both many to many and many to one constantly updates the hidden state by the inputs. Many to many produces output every tick while many to one has only one output at the last hidden state. The last hidden state can be seen as summarizing the entire input sequence.  


* On one to many, the input is commonly used to initialize the initial hidden state and the RNN ticks itself through the graph, producing output in the process.  


* Sequence to sequence is a combination of many to one(encoder) and one to many(decoder). The output of encoder summarizes the whole input sequence, and each output of the decoder is a prediction of which word to use using that summary.  

## Character-level language model sampling
* A letter is represented with a vector, which its dimension is the total number of possible characters.  

* At training, the networks is provided one input at a time. Its target is to create a softmax prediction of the next character, and an svm loss is produced each time.
![CLMtrain](images/CLMtrain.png)
<br> <br>
* At test time, only one initial character is given. The network then **samples** a character from the prediction and uses it as the next input.
    * Why sample and not pick the one with highest score? - It gives more diversity. The RNN can give multiple reasonable outputs depending on its initial state.
    * Why input a strict vector(all zero except one) and not the softmax probability right before? - Softmax input would be very different from what it saw on training time. It also takes much more computation since it has to perform full matrix multiplication while a strict vector doesn't have to.
![CLMtest](images/CLMtest.png)
<br> <br>
* Backpropagation should technically be done after going through all inputs, by going back through all inputs in reverse. However with inputs so large, this is infeasible.
![Backprop](images/backpropthrutime.png)
<br> <br>
* So we'll backpropagate every certain batch of inputs, not re-computing the hidden state from the start. This is an approximation of true gradients, thus the name **truncated backpropagation through time**.
![Truncate](images/truncatedBackprop.png)
[Source](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)
<br> <br>
* And somehow these seem to work super well! And even though we have only told them to predict what would come next, some cells have managed to develop unique abilities such as quote detection, line length tracking, and code depth tracking, 

## Image captioning
* The vector before last FC layer in CNN contains a good feature representation of the image.
* We can connect that vector to RNN and make a caption for that image.
* One way to implement this is to add that to our original internal state formula. So  

$$
h_t = tanh(W_{xh}x + W_{hh}h + W_{ij}v)
$$
where $v$ is the vector from CNN.

* Here the vocabulary would contain most of the words in English, along with START,END tokens. START token indicates the start of caption, and would be the first input to the RNN. END token works like a period. When the RNN generates END token, the captioning ends.

* The network is completely trained with supervision, datasets with image and captions such as COCO are used. When training, we can also pass the gradient to the final layer of CNN, to "jointly tune" all parts of the model for this task.


* As impressive as their results are, it fails quite badly when given inputs distant from the training set.

## Image captioning with attention
* The model can steer its attention to certain parts of the image for captioning.
* The vector $v$ is now a grid of multiple vectors that summarizes a local part of a image.
* The RNN makes an additional output a distribution vector to focus or ignore certain locations of the image. Each vector in $v$ gets weighted by the distribution vector, and the new weighted $v$ becomes the new input to the next hidden state.

## Multilayer RNN
* Stack multiple layers of hidden states! The formula for hidden state in $l$th layer at time $t$ would be
$$
h^l_t = tanh W^l \left( \begin{matrix} h^{l-1}_t \\ h^l_{t-1} \end{matrix} \right)
$$
<br>
* somewhat like deep CNN, but 2-3 layers is enough here.

## Gradient problem
* At forward pass, the same $W$ is multiplied multiple times to produce $h_t$. This means that the same thing($W^T$) will happen to backprop, possibly exploding or killing the gradient.


* Exploding gradients
    * When largest singular value > 1
    * Use gradient clipping - If the L2 norm is above threshold, scale the whole gradient.


* Vanishing gradients
    * When largest singuar value < 1
    * Motivation to LSTM

# 🤔
* Singular value nooo

## LSTM
* Alleviates the vanishing gradient problem.
* Invented in 1997. WHY ARE PEOPLE SO SMART
* The next hidden state $h_t$ is now determined by 4 different gates, $i, f, o, g$.

$$
\left( \begin{matrix} i\\f\\o\\g \end{matrix} \right) =
\left( \begin{matrix} \sigma \\ \sigma \\ \sigma \\ tanh \end{matrix} \right)
W \left( \begin{matrix} h_{t-1} \\ x_t \end{matrix} \right)
$$
now the $W$ matrix is (4h x 2h) instead of (h x 2h), where h is the dimension of $x$ and $h$
* These 4 gates create the $h_t$ via a new cell state $c_t$.  

$$
\begin{matrix}
c_t & = & f \odot c_{t-1} + i \odot g \\
h_t & = & o \odot tanh(c_t)
\end{matrix}
$$
where $\odot$ indicates elementwise multiplication.
* This becomes more intuitive once we reailze that $i,f,o$ is in range (0,1) and $g$ is in (-1,1).
* $f$ lets the cell to _forget_ what's been before, and $o$ determines how much to _reveal_ itself to the hidden state. $i$ and $g$ indicates how much it wants to _write_ on previous memory, and $g$ determines its sign. 


* Now in terms of gradient, we need the gradient of $c_t$ since that's what controls $h_t$. And to compute the gradient of $c_{t-1}$, all we have to do is elementwise-multiply $f$.
* Because it's **elementwise multiplication** and $f$ **is different for every timestep**, this helps solve the vanishing gradient problem.
* Also because LSTM gradient passes through **tanh only once**(for local gradient of $o$), the gradient is more easily preserved.

![LSTM](images/LSTMblock.png)

* It still is possible to have vanishing gradients if $f$ constantly has low values. But again, since $f$ is different everytime, it is less prone to happen. People like to initialize $f$ with positive values to address this bug a bit.  

# 🤔
* GRU?